In [17]:
import json
import numpy as np
import pandas as pd
from pprint import pprint
from collections import defaultdict

In [25]:
vistext_test_captions = np.load("./vistext_test_id_to_captions.npy", allow_pickle=True).item()
matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
gpt4v_L3_captions = np.load("./vistext_test_id_to_L3_gpt4v_captions.npy", allow_pickle=True).item()
gpt4v_L3_alt_captions = np.load("./vistext_test_id_to_L3_gpt4v_alt_captions.npy", allow_pickle=True).item()
gpt4v_L3_table_captions = np.load("./vistext_test_id_to_L3_gpt4v_table_captions.npy", allow_pickle=True).item()
gpt4v_L4_captions = np.load("./vistext_test_id_to_L4_gpt4v_captions.npy", allow_pickle=True).item()
gpt4v_L4_alt_captions = np.load("./vistext_test_id_to_L4_gpt4v_alt_captions.npy", allow_pickle=True).item()
gpt4v_L4_table_captions = np.load("./vistext_test_id_to_L4_gpt4v_table_captions.npy", allow_pickle=True).item()

In [26]:
vistext_id_to_combined_captions = []
for image_id, caption_dict in vistext_test_captions.items():
    image_dict = {}
    image_dict["image_id"] = image_id
    image_dict["human"] = [hc["L1"] + " " + hc["L2L3"] for hc in caption_dict]
    if image_id in matplotalt_captions:
        image_dict["heuristic"] = [matplotalt_captions[image_id]]
    if image_id in gpt4v_L4_captions:
        image_dict["gpt-4-turbo-L4"] = [gpt4v_L4_captions[image_id]]
    if image_id in gpt4v_L4_alt_captions:
        image_dict["gpt-4-turbo-alt-L4"] = [gpt4v_L4_alt_captions[image_id]]
    if image_id in gpt4v_L4_table_captions:
        table_caption = gpt4v_L4_table_captions[image_id]
        table_idx = table_caption.lower().find("\n\ndata table:")
        if table_idx != -1:
            table_caption = table_caption[:table_idx]
        image_dict["gpt-4-turbo-table-L4"] = [table_caption]
    if image_id in gpt4v_L3_captions:
        image_dict["gpt-4-turbo-L3"] = [gpt4v_L3_captions[image_id]]
    if image_id in gpt4v_L3_alt_captions:
        image_dict["gpt-4-turbo-alt-L3"] = [gpt4v_L3_alt_captions[image_id]]
    if image_id in gpt4v_L3_table_captions:
        table_caption = gpt4v_L3_table_captions[image_id]
        table_idx = table_caption.lower().find("\n\ndata table:")
        if table_idx != -1:
            table_caption = table_caption[:table_idx]
        image_dict["gpt-4-turbo-table-L3"] = [table_caption]
    vistext_id_to_combined_captions.append(image_dict)

combined_captions_df = pd.DataFrame.from_dict(vistext_id_to_combined_captions)
combined_captions_df.to_json("./vistext_id_to_combined_captions.jsonl", orient='records', lines=True)